<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#How-to-manage-meshes" data-toc-modified-id="How-to-manage-meshes-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>How to manage meshes</a></span><ul class="toc-item"><li><span><a href="#2D-case" data-toc-modified-id="2D-case-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>2D case</a></span></li><li><span><a href="#3D-case" data-toc-modified-id="3D-case-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>3D case</a></span></li></ul></li></ul></div>

# How to manage meshes

F. Dubois 2016

## 2D case

In [ ]:
# Initialization
import numpy as np
from pylmgc90 import pre
# 2D
dim = 2

In [ ]:
# step by step
m1 = pre.mesh(dimension=2)
m1.addNode( pre.node(np.array([0.,0.]), number=1) )
m1.addNode( pre.node(np.array([1.,0.]), number=2) )
m1.addNode( pre.node(np.array([0.,1.]), number=3) )
m1.addNode( pre.node(np.array([1.,1.]), number=4) )

# adding a 2D element with a group
m1.addBulk( pre.element(dim, [1,2,4,3], physicalEntity='all'))

# adding a 1D elements with groups ; beware clockwise gives outside normal
m1.addBulk( pre.element(dim-1,[2,1],physicalEntity='down'))
m1.addBulk( pre.element(dim-1,[1,4],physicalEntity='left'))
m1.addBulk( pre.element(dim-1,[4,3],physicalEntity='up'))
m1.addBulk( pre.element(dim-1,[3,2],physicalEntity='right'))

# what are these things ?
#help(pre.mesh)
#help(pre.node)
#help(pre.element)
#help(m1.addBulk)

In [ ]:
# using builtin function to mesh
m2 = pre.buildMesh2D('Q4', x0=1.5, y0=1.5, lx=1., ly=1., nb_elem_x=10, nb_elem_y=10)

# what is doing buildMesh2D ?
#help(pre.buildMesh2D)

# some introspection
#dir(m2)
#print(m2.nodes.values()[0].coor)

In [ ]:
!gmsh ./gmsh_files/block.msh

In [ ]:
# loading a gmsh file
m3 = pre.readMesh('./gmsh_files/block.msh',dim)

# what is doing readMesh ?
#help(pre.readMesh)

In [ ]:
# a mesh is not an avatar
# lets build an avatar; we need a model and a material
mo = pre.model(name='M2D_L',physics='MECAx',element='Q4xxx',dimension=2, external_model='MatL_',
               kinematic='small',material='elas_',anisotropy='iso__',mass_storage='coher')
ma = pre.material(name='steel',materialType='ELAS',elas='standard',
                  young=0.1e+15,nu=0.2,anisotropy='isotropic',
                  density=0.25e+4)

if True:
    #possible options:
    print(pre.config.lmgc90dicts.bulkBehavOptions.keys())
    # see parameters for 'ELAS'
    print('options for ELAS:')
    print(pre.config.lmgc90dicts.bulkBehavOptions['ELAS'])

    #possible options:
    print(pre.config.lmgc90dicts.modelOptions.keys())
    # see parameters for 'MECAx'
    print('options for MECAx:')
    print(pre.config.lmgc90dicts.modelOptions['MECAx'])

In [ ]:
# now we can build avatars
b1 = pre.buildMeshedAvatar(mesh=m1, model=mo, material=ma)
b2 = pre.buildMeshedAvatar(mesh=m2, model=mo, material=ma)
b3 = pre.buildMeshedAvatar(mesh=m3, model=mo, material=ma)
b3.translate(dx=3.,dy=3.)

In [ ]:
# Lets draw
# put everything in a container
bodies  = pre.avatars()
bodies += b1
bodies += b2
bodies += b3

# draw it 
pre.visuAvatars(bodies)

## 3D case 

In [ ]:
# Initialization
import numpy as np
from pylmgc90 import pre

dim=3

In [ ]:
# step by step
m1 = pre.mesh(dimension=dim)
m1.addNode( pre.node(np.array([0.,0.,0.]), number=1) )
m1.addNode( pre.node(np.array([1.,0.,0.]), number=2) )
m1.addNode( pre.node(np.array([1.,1.,0.]), number=3) )
m1.addNode( pre.node(np.array([0.,1.,0.]), number=4) )
m1.addNode( pre.node(np.array([0.,0.,1.]), number=5) )
m1.addNode( pre.node(np.array([1.,0.,1.]), number=6) )
m1.addNode( pre.node(np.array([1.,1.,1.]), number=7) )
m1.addNode( pre.node(np.array([0.,1.,1.]), number=8) )

m1.addBulk( pre.element(dim, [1,2,3,4,5,6,7,8], physicalEntity='all'))


In [ ]:
# using builtin function to mesh
m2 = pre.buildMeshH8(x0=1.5, y0=1.5, z0=1.5, lx=1., ly=1., lz=1., nb_elem_x=10, nb_elem_y=10, nb_elem_z=10)

In [ ]:
# loading a gmsh file
m3 = pre.readMesh('./gmsh_files/Donut.msh',dim)

In [ ]:
# a mesh is not an avatar
#first create model & material
moH8 = pre.model(name='M3D_L',physics='MECAx',element='H8xxx',dimension=3, external_model='MatL_',
                 kinematic='small',material='elas_',anisotropy='iso__',mass_storage='coher')
moT4 = pre.model(name='M3D_L',physics='MECAx',element='TE4xx',dimension=3, external_model='MatL_',
                 kinematic='small',material='elas_',anisotropy='iso__',mass_storage='coher')

ma = pre.material(name='steel',materialType='ELAS',elas='standard',
                  young=0.1e+15,nu=0.2,anisotropy='isotropic',
                  density=0.25e+4)
#then create avatar
b1 = pre.buildMeshedAvatar(mesh=m1, model=moH8, material=ma)
b2 = pre.buildMeshedAvatar(mesh=m2, model=moH8, material=ma)
b3 = pre.buildMeshedAvatar(mesh=m3, model=moT4, material=ma)
b3.translate(dx=3.,dy=3.,dz=3.)

In [ ]:
# Lets draw
bodies = pre.avatars()
bodies+=b1
bodies+=b2
bodies+=b3
pre.visuAvatars(bodies)